In [1]:
from typing import Optional

import functools

import numpy as np

import ml_collections
import tensorflow as tf
import tensorflow_io as tfio

import tokenizer
import sequence_packing
import random
import os

2024-04-25 05:57:53.292652: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 05:57:53.435470: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 05:57:53.436841: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 05:57:54.837877: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
def loadjson_and_rekey(ds):
    """normalization with key mapping"""
    json_specs = {
        "text": tf.TensorSpec(tf.TensorShape([]), tf.string, name="text"),
    }
    key_map={"inputs": None, "targets": "text"}
    text_max_len = 10 * 1024 * 1024 # 10M

    def _loadjson_and_rekey(x, json_specs, key_map=None):
        """Replace the feature keys according to the mapping in `key_map`.
        For example, if the dataset returns examples of the format:
        {'foo': 'something', 'bar': 'something else', 'zoo': 'others'}
        and key_map = {'boo': 'foo', 'spar': 'bar', 'zoo': None} then this function will return
        examples with the format
        {'boo': 'something', 'spar': 'something else'}
        If a mapping is to None, then the key will be dropped.
        Args:
          x: an example to process.
          key_map: dictionary mapping new keys to original keys
        Returns:
          A preprocessed example with the format listed above.
        """
        x = tfio.experimental.serialization.decode_json(x, specs=json_specs)
        x["text"] = tf.strings.substr(x["text"], 0, text_max_len, unit='BYTE')

        x = {
            new_key: x[old_key] for new_key, old_key in key_map.items() if old_key
        }

        return x

    return ds.map(
        functools.partial(_loadjson_and_rekey, json_specs=json_specs, key_map=key_map), num_parallel_calls=AUTOTUNE
    )


def split_tokens(
    dataset,
    max_tokens_per_segment=128,
    feature_key="targets",
):
    """Split examples into multiple examples each.
    The intended use case is to break up long examples for use in unsupervised
    transfer-learning.
    This function is generally preceded by select_random_chunk.
    Args:
      dataset: a tf.data.Dataset with dictionaries containing the key feature_key.
      max_tokens_per_segment: an integer, the maximum number of tokens in each
        segment. Only the final segment may be shorter.
      feature_key: a string, the feature to split
    Returns:
      a dataset
    """

    def _split_tokens(x):
        """Split one token sequence into multiple multiple."""
        tokens = x[feature_key]
        n_tokens = tf.size(tokens)
        length = max_tokens_per_segment

        # Pad to a multiple of length, then use tf.reshape to split up the tokens
        # into num_segments segments each of the given length.
        num_segments = tf.cast(
            tf.math.ceil(tf.cast(n_tokens, tf.float32) / tf.cast(length, tf.float32)),
            tf.int32,
        )
        padding = num_segments * length - tf.size(tokens)
        tokens = tf.pad(tokens, [[0, padding]])
        return tf.reshape(tokens, [-1, length])

    def _strip_padding(x):
        return {feature_key: tf.boolean_mask(x, tf.cast(x, tf.bool))}

    # Filter empty examples.
    dataset = dataset.filter(lambda x: tf.not_equal(tf.size(x[feature_key]), 0))
    dataset = dataset.map(_split_tokens, num_parallel_calls=AUTOTUNE)
    dataset = dataset.unbatch()
    return dataset.map(_strip_padding, num_parallel_calls=AUTOTUNE)


def split_tokens_to_targets_length(dataset, sequence_length):
    return split_tokens(dataset, max_tokens_per_segment=sequence_length)


def load_base_dataset(
    pattern,
    seed,
):
    data_paths = sorted(tf.io.gfile.glob(pattern))

    # shard dataset now
    print((pattern, "all_file_count", len(data_paths)))
    data_num_shards = 1
    data_index = 0
    data_paths = [
        d
        for i, d in enumerate(data_paths)
        if i % data_num_shards == data_index
    ]
    random.seed(seed + data_index)
    random.shuffle(data_paths)

    print((pattern, "share_file_count", data_num_shards, data_index, len(data_paths), data_paths[:2]))

    ds = tf.data.TextLineDataset(
        data_paths,
        compression_type="GZIP",
        buffer_size=8 * 1024 * 1024,
        num_parallel_reads=2,
    )

    return ds

In [4]:
dataset_path = "/home/genggui001/gdrive/gg-nlp-lm-new-3"
data_shuffle_seed = 1234
max_target_length = 4096

In [ ]:
zh_ds = load_base_dataset(
    pattern=os.path.join(dataset_path, "gg_zh/**/*.jsonl.gz"),
    seed=data_shuffle_seed,
)


In [5]:
"""Load and return dataset of batched examples for use during training."""
en_ds = load_base_dataset(
    pattern=os.path.join(dataset_path, "gg_en/**/*.jsonl.gz"),
    seed=data_shuffle_seed,
)

zh_ds = load_base_dataset(
    pattern=os.path.join(dataset_path, "gg_zh/**/*.jsonl.gz"),
    seed=data_shuffle_seed,
)

other_ds = load_base_dataset(
    pattern=os.path.join(dataset_path, "uonlp_culturax_shuffle/**/*.jsonl.gz"),
    seed=data_shuffle_seed,
)

code_ds = load_base_dataset(
    pattern=os.path.join(dataset_path, "the-stack-dedup/**/*.jsonl.gz"),
    seed=data_shuffle_seed,
)

train_ds = tf.data.Dataset.sample_from_datasets(
    datasets = [
        en_ds.repeat(),
        zh_ds.repeat(),
        other_ds.repeat(),
        code_ds.repeat(),
    ], 
    weights=[
        0.45,
        0.2,
        0.25,
        0.1,
    ],
    seed=data_shuffle_seed,
)

train_ds = loadjson_and_rekey(
    train_ds, 
)

('/home/genggui001/gdrive/gg-nlp-lm-new-3/gg_en/**/*.jsonl.gz', 'all_file_count', 2088)
('/home/genggui001/gdrive/gg-nlp-lm-new-3/gg_en/**/*.jsonl.gz', 'share_file_count', 1, 0, 2088, ['/home/genggui001/gdrive/gg-nlp-lm-new-3/gg_en/WebText-en/chunk-00029.jsonl.gz', '/home/genggui001/gdrive/gg-nlp-lm-new-3/gg_en/falcon-refinedweb/chunk-00497.jsonl.gz'])
('/home/genggui001/gdrive/gg-nlp-lm-new-3/gg_zh/**/*.jsonl.gz', 'all_file_count', 873)
('/home/genggui001/gdrive/gg-nlp-lm-new-3/gg_zh/**/*.jsonl.gz', 'share_file_count', 1, 0, 873, ['/home/genggui001/gdrive/gg-nlp-lm-new-3/gg_zh/TeleChat-PTD/chunk-00292.jsonl.gz', '/home/genggui001/gdrive/gg-nlp-lm-new-3/gg_zh/WebText-cn/chunk-00153.jsonl.gz'])
('/home/genggui001/gdrive/gg-nlp-lm-new-3/uonlp_culturax_shuffle/**/*.jsonl.gz', 'all_file_count', 5000)
('/home/genggui001/gdrive/gg-nlp-lm-new-3/uonlp_culturax_shuffle/**/*.jsonl.gz', 'share_file_count', 1, 0, 5000, ['/home/genggui001/gdrive/gg-nlp-lm-new-3/uonlp_culturax_shuffle/s/schunk-02004

2024-04-25 05:58:21.888689: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX AVX2 AVX512F FMA
2024-04-25 05:58:21.893814: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


In [6]:
from sentencepiece import SentencePieceProcessor

tokenize_processor = SentencePieceProcessor()
tokenize_processor.Load("/home/genggui001/code/maxtext/assets/llama_add_world.model")


def tokenize_numpy_fn(t: np.ndarray):
    text = t.decode('utf-8')
    token_ids = tokenize_processor.EncodeAsIds(text)
    token_ids = [tokenize_processor.bos_id()] + token_ids + [tokenize_processor.eos_id()]
    return np.asarray(token_ids, dtype=np.int32)

train_ds = train_ds.shuffle(128, seed=data_shuffle_seed)
train_ds = train_ds.map(
    lambda x: {
        'targets': tf.numpy_function(
            func=tokenize_numpy_fn, 
            inp=[x['targets']], 
            Tout=tf.int32, 
            stateful=False,
        )
    },
    num_parallel_calls=AUTOTUNE,
)
train_ds = split_tokens_to_targets_length(train_ds, max_target_length+1)
train_ds = train_ds.map(
    lambda x: {
        "inputs": x['targets'][:-1],
        "targets": x['targets'][1:],
    }, 
    num_parallel_calls=AUTOTUNE
)
train_ds = sequence_packing.pack_dataset(train_ds, max_target_length)


train_ds = train_ds.batch(
    8, 
    drop_remainder=True
)

train_ds = train_ds.prefetch(32)

In [7]:
iterator = train_ds.as_numpy_iterator()

In [8]:
ckpt = tf.train.Checkpoint(iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, '/home/genggui001/code/maxtext/tmp/tf_ckpts', max_to_keep=3)

In [9]:
ckpt.restore(manager.latest_checkpoint)

In [10]:
from tqdm.auto import tqdm

/home/genggui001/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
for _ in tqdm(range(64)):
    tmp = next(iterator)

100%|██████████| 64/64 [01:09<00:00,  1.08s/it]


In [12]:
manager.save(64)

'/home/genggui001/code/maxtext/tmp/tf_ckpts/ckpt-64'

In [20]:
for item in tmp['inputs_position'][2]:
    print(item)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27